# Линейная регрессия

На сегодняшнем занятии будем работать с датасетом BlackFriday (ниже есть его описание):

Description
The dataset here is a sample of the transactions made in a retail store. The store wants to know better the customer purchase behaviour against different products. Specifically, here the problem is a regression problem where we are trying to predict the dependent variable (the amount of purchase) with the help of the information contained in the other variables.

Classification problem can also be settled in this dataset since several variables are categorical, and some other approaches could be "Predicting the age of the consumer" or even "Predict the category of goods bought". This dataset is also particularly convenient for clustering and maybe find different clusters of consumers within it.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%pylab inline

In [ ]:
data = pd.read_csv('BlackFriday.csv')
data.head()

# Подготовка данных

In [ ]:
missing_values = data.isnull().sum().sort_values(ascending=False)
missing_values = missing_values[missing_values > 0] / data.shape[0]
print(f'{missing_values * 100} %')

Скорее всего NaN-ы в Product_Category_2 и Product_Category_3 означают, что человек не покупал продукты из этих категорий.

Поэтому можем заменить эти пропущенные значения на 0.

In [ ]:
data = data.fillna(0)

In [ ]:
missing_values = data.isnull().sum().sort_values(ascending=False)
missing_values = missing_values[missing_values > 0] / data.shape[0]
print(f'{missing_values *100} %')

Посмотрим, какие типы данных есть в нашем датасете. 

Выведите на экран тип каждой колонки в таблице.

In [ ]:
data.info()

Обработаем нечисловые колонки.

1) Выведите на экран все возможные значения из столбца Gender.

2) Замените значение 'M' на 1, а 'F' на 0.

In [ ]:
# your code here

Поработаем со столбцом Age.

1) Выведите на экран все возможные значения из столбца Age.

2) Напишите функцию для перевода каждого диапазона возрастов в число (самые маленькие возрасты - 0, следующий диапазон - 1 и т.д.). Замените значения в столбце Age на числовые, применив эту функцию.

In [ ]:
#your code here

Обработаем столбец City_Category.

1) Выведите все уникальные значения данного столбца.

2) Закодируйте столбец с помощью OneHotEncoding (get_dummies).

In [ ]:
print(data['City_Category'].unique())

data = pd.get_dummies(data, columns=['City_Category'], drop_first=True, dtype = int)
data

Наконец, обработаем Stay_In_Current_City_Years.

1) Выведите на экран все уникальные значения данного столбца.

2) Замените '4+' на 4, а все остальные значения оставьте как есть, но переведите их в числовой вид (x -> int(x)).

In [ ]:
#your code here   

In [ ]:
data.head()

Удалим столбцы User_ID и Product_ID.

In [ ]:
#your code here

# Исследование данных

1. Как влияет пол на целевую переменную Purchase? Постройте диаграмму.

2. Как влияет возраст на целевую переменную Purchase? Постройте диаграмму.

3. Как влияет City_Category на целевую переменную Purchase? Постройте диаграмму.

Нарисуйте матрицу корреляций признаков.

In [ ]:
plt.figure(figsize=(12,9))

corr = data.corr()
sns.heatmap(corr, annot=True, linewidths=.5, fmt= '.2f', mask=np.zeros_like(corr, dtype=np.bool), \
            cmap=sns.diverging_palette(100,200, as_cmap=True), square=True)

# Построение модели

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

X = data.copy()

y = data['Purchase']
del X['Purchase']

ind_train, ind_test = train_test_split(range(len(X)), test_size=0.2, random_state=111)
X_train = X.iloc[ind_train]
y_train = y[ind_train]
X_test = X.iloc[ind_test]
y_test = y[ind_test]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
print(y_pred)

In [ ]:
print(r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred))

Добавим L1 регуляризацию.

In [ ]:
from sklearn.linear_model import Lasso

for a in np.arange(0.1, 100.1, 25):
    regressor2 = Lasso(alpha=a)
    regressor2.fit(X_train, y_train)

    y_pred2 = regressor2.predict(X_test)

    print('alpha={}'.format(a))
    print('Scores:', r2_score(y_test,y_pred2), mean_squared_error(y_test, y_pred2))
    print(regressor2.coef_,'\n')

**Видим, что L1-регуляризация зануляет некоторые веса и таким образом производит отбор признаков.**

Подберем оптимальное значение параметра регуляризации по кросс-валидации

In [ ]:
from sklearn.linear_model import LassoCV

n_alphas = 200
alphas = np.linspace(0.1, 10, n_alphas)

lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42)
lasso_cv.fit(X, y)

In [ ]:
lasso_cv.coef_

In [ ]:
lasso_cv.alpha_

In [ ]:
regressor2 = Lasso(alpha=lasso_cv.alpha_)

regressor2.fit(X_train, y_train)

y_pred2 = regressor2.predict(X_test)

print('Scores:', r2_score(y_test, y_pred2), mean_squared_error(y_test, y_pred2))

Добавим L2 регуляризацию.

In [ ]:
from sklearn.linear_model import Ridge

for a in np.arange(-10, 10.1, 2.5):
    regressor3 = Ridge(alpha=a)
    regressor3.fit(X_train, y_train)

    y_pred3 = regressor3.predict(X_test)

    print('alpha={}'.format(a))
    print('Scores:',r2_score(y_test, y_pred3), mean_squared_error(y_test, y_pred3))
    print(regressor3.coef_)

**Ridge регрессия не зануляет веса (они могут быть очень маленькими, но ненулевыми).**

Теперь подберем оптимальное a по кросс-валидации.

In [ ]:
from sklearn.linear_model import RidgeCV

n_alphas = 20
ridge_alphas = np.logspace(-2, 6, n_alphas)

ridge_cv = RidgeCV(
    alphas=ridge_alphas, 
    scoring='neg_mean_squared_error',
    cv=5,
)
ridge_cv.fit(X, y)

In [ ]:
ridge_cv.alpha_

In [ ]:
regressor3 = Lasso(alpha=ridge_cv.alpha_)

regressor3.fit(X_train, y_train)

y_pred3 = regressor3.predict(X_test)

print('Scores:', r2_score(y_test, y_pred3), mean_squared_error(y_test, y_pred3))

Добавим одновременно и L1, и L2 регуляризацию.

У ElasticNet два параметра: alpha и l1_ratio.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

In [ ]:
from sklearn.linear_model import ElasticNet

#your code here

Посмотрим на веса, которые присвоила линейная регрессия признакам.

In [ ]:
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(regressor.coef_))], axis = 1)
coefficients

# Feature Ingineering

Попробуйте соорудить новые признаки.

Идеи:

1) Группировка исходных признаков по бинам
(попробуйте сгруппировать числовые признаки, например, Product_Category_1(2,3))

2) Перемножение признаков, которые в паре могут влиять на результат (линейная регрессия не умеет умножать)

3) Взятие других функций от признаков.

4) Счётчики.

In [ ]:
def code_mean(data, cat_feature, real_feature):
    """
    Возвращает словарь, где ключами являются уникальные категории признака cat_feature, 
    а значениями - средние по real_feature
    """
    return dict(data.groupby(cat_feature)[real_feature].mean())

In [ ]:
data['age_average'] = list(map(code_mean(data.iloc[ind_train], 'Age', "Purchase").get, data.Age))